In [1]:
import mysql.connector as mariadb
from mysql.connector import Error
import numpy as np
import pandas as pd
from getpass import getpass



In [2]:
import sys
import os
import findspark
findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [2]:
#read the database username and password from secret.txt
secrets_file = os.path.join("files", "secret.txt")
with open(secrets_file, "r") as f:		
    lines = f.readlines()
for line in lines:
    words = line.split("=")
    if (words[0] == "user"):
        user = words[1].strip()
    elif (words[0] == "password"):
        password = words[1].strip()
f.close()


In [3]:
def get_database_connection():
    connection = mariadb.connect(host='localhost',
                                            database='project_db',
                                            user=user,
                                            password=password)
    if connection.is_connected():
        db_Info = connection.get_server_info()
    #cursor = connection.cursor()
    return(connection)




    1)    Used to display the transactions made by customers living in a given zip code for a given month and year.
                                        Order by day in descending order.


In [6]:
def total_customers():
    
    connection = get_database_connection()
    cursor = connection.cursor()

    flag = True
    zip_code = input("Enter Your Zip Code:  ")
    while (flag):
        if not zip_code.isdigit():
            zip_code = input('enter a valid zipcode')
        else:
            flag=False
    
    transaction_year = input("Enter the year for transactions records:  ")
    flag=True
    while (flag):
        if not transaction_year.isdigit() or len(transaction_year)!=4:
            transaction_year = input("Enter valid year format XXXX")
        else:
            flag = False
        
    transaction_month = input("Enter the month in the form (XX): ").zfill(2)
    flag = True
    while (flag):
        if  not transaction_month.isdigit():
            transaction_month = input('Please enter valid month')
        else:
            flag=False
    

    
    query ="SELECT CUST_CC_NO,date_format(TIMEID,'%Y-%m-%d') as transaction_date,CUST_SSN,BRANCH_CODE, \
    TRANSACTION_TYPE,TRANSACTION_VALUE,TRANSACTION_ID \
    FROM (SELECT ssn   from cdw_sapp_customer  where cust_zip = %s) as customer \
    join cdw_sapp_credit_card on cdw_sapp_credit_card.cust_ssn = customer.ssn\
    where SUBSTRING(timeid, 1, 4)  = %s and SUBSTRING(timeid, 5, 2)  = %s \
    Group by timeid desc;"

    cursor.execute(query,(zip_code,transaction_year,transaction_month,))
    records = cursor.fetchall()
    print("\n List of  customer transactions in a given Zip code : ")
    print("\n\n Customer no\t\tdate\t   cust_ssn  code  type       value  id")
    for row in records:
        print(row)
    connection.close()
total_customers()

Enter Your Zip Code:  39120
Enter the year for transactions records:  2018
Enter the month in the form (XX): 08

 List of  customer transactions in a given Zip code : 


 Customer no		date	   cust_ssn  code  type       value  id
('4210653362697310', '2018-08-28', 123459262, 124, 'Grocery', 59.07, 3544)
('4210653362697310', '2018-08-27', 123459262, 164, 'Grocery', 97.12, 3547)
('4210653396189363', '2018-08-25', 123457139, 42, 'Gas', 98.24, 15191)
('4210653374740577', '2018-08-24', 123459564, 180, 'Education', 95.32, 2194)
('4210653396189363', '2018-08-23', 123457139, 17, 'Gas', 23.02, 15222)
('4210653323781193', '2018-08-22', 123455582, 135, 'Test', 36.19, 22991)
('4210653359877473', '2018-08-21', 123457519, 135, 'Grocery', 100.54, 12635)
('4210653310061055', '2018-08-20', 123456100, 54, 'Gas', 4.22, 20549)
('4210653310061055', '2018-08-19', 123456100, 107, 'Education', 46.14, 20536)
('4210653374740577', '2018-08-18', 123459564, 178, 'Gas', 4.96, 2172)
('4210653387378802', '2018-08-17',

                    2) Used to display the number and total values of transactions for a given type.


In [8]:
def transaction_value():
    transaction_type = input("Enter The Type Of Transaction :    ")
    flag = True
    connection = get_database_connection()
    cursor = connection.cursor()

    query = "select distinct(transaction_type) from cdw_sapp_credit_card"
    
    cursor.execute(query,)
    records = cursor.fetchall()
    

    type_lst =[]
    for row in records:
        type_lst += row;
        
    
    while flag:
        if transaction_type.title() not in type_lst:
            transaction_type =input('Invalid transaction_type, Re enter the transaction type:'   )
        else:
            flag=False

    query = "SELECT count(*),sum(transaction_value) from cdw_sapp_credit_card where transaction_type = %s"

    cursor.execute(query,(transaction_type,))
    records = cursor.fetchall()
    print("\n Number of transactions and total value for transaction type " + transaction_type + " : ")
    print(records)
    connection.close()
    
transaction_value()


Enter The Type Of Transaction :    gas

 Number of transactions and total value for transaction type gas : 
[(6605, 336059.26000000036)]


        3)    Used to display the number and total values of transactions for branches in a given state.

In [10]:
def transaction_value_branch():
    connection = get_database_connection()
    cursor = connection.cursor()
    
    flag = True
    query = "select distinct(branch_state) from cdw_sapp_branch" 
    cursor.execute(query,)
    records = cursor.fetchall()
    branch_state = input("Enter the state in XX format :    ")
    state_lst =[]
    for row in records:
        state_lst += row;
    while flag:
        if branch_state.upper() not in state_lst:
            branch_state =input('Invalid state, Re enter the state value in the format(XX):'   )
        else:
            flag=False
        
    
    query="select count(transaction_type) , sum(transaction_value) from cdw_sapp_credit_card \
                   join cdw_sapp_branch on cdw_sapp_branch.branch_code = cdw_sapp_credit_card.BRANCH_CODE \
                   where cdw_sapp_branch.branch_state = %s \
                   group by cdw_sapp_branch.branch_code;"
    cursor.execute(query,(branch_state,))
    records = cursor.fetchall()
    for row in records:
        print(row)
        
transaction_value_branch()
        
        






Enter the state in XX format :    mn
(1122, 54472.709999999934)
(1143, 58052.48999999997)
(1116, 55934.67000000016)


                                    2.2 Customer Details Module

                            1) Used to check the existing account details of a customer.

In [ ]:
def get_input():
    
    flag = True
        
    ssn = getpass("Enter the last 4 digits :    " )
    while flag:
        if not ssn.isdigit() or len(ssn) != 4:
            ssn =getpass('Invalid ssn, Re enter the ssn without -:    ')
        else:
            flag=False
            
    lastname = input('Enter the lastname:   ')
    return()


In [ ]:
def customer_details():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
    flag = True
        
    ssn = getpass("Enter the last 4 digits :    " )
    while flag:
        if not ssn.isdigit() or len(ssn) != 4:
            ssn =getpass('Invalid ssn, Re enter the ssn without -:    ')
        else:
            flag=False
            
    lastname = input('Enter the lastname:   ')
    
    query="select * from cdw_sapp_customer where substr(ssn,6,4) = %s and Last_name = %s"
    
    cursor.execute(query,(ssn,lastname,))
    
    records = cursor.fetchall()
    
    if len(records) == 0:
        print('Could find any account details with given input')
    else:
    
        print("SSN:\t\t"+ '***-**-'+ssn)
        print("Name:\t\t"+(records[0][1]+' '+records[0][2].title()+' '+records[0][3]))
        print("CC No:\t\t"+records[0][4])
        print("Address:\t" + records[0][5])
        print("\t\t" + records[0][6])
        print("\t\t" + records[0][7]+','+records[0][8]+' - '+str(records[0][9]))
        print("Ph No:\t\t" + records[0][10])
        print("Email:\t\t" + records[0][11])


    cursor.close()    
    
customer_details()


In [ ]:
# def number_to_string(argument):
#     match argument:
#         case 0:
#             return "zero"
#         case 1:
#             return "one"
#         case 2:
#             return "two"
#         case default:
#             return "something"
 
 
# if __name__ = "__main__":
#     argument = 0
#     number_to_string(argument)


                    2) Used to modify the existing account details of a customer.

In [ ]:
def modify_customer_details():
    customer_details()
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
    flag = True
        
    ssn = getpass("Enter the last 4 digits :    " )
    while flag:
        if not ssn.isdigit() or len(ssn) != 4:
            ssn =getpass('Invalid ssn, Re enter the ssn without -:    ')
        else:
            flag=False
            
    lastname = input('Enter the lastname:   ')
    query="select * from cdw_sapp_customer where substr(ssn,6,4) = %s and Last_name = %s"
    
    
    
    cursor.execute(query,(ssn,lastname,))
    
    records = cursor.fetchall()
    
    print("Procced if you want to make changes to your account: y/n")
    a,b, = records[0][5].split(',')
    print(a,b)
#     choice=input()
#     if choice.lower() == 'y':
#         apptno,street=input(("Address:\t" + records[0][5]) or records[0][5].split(','))
#         if (apptno == Null):
#             print("keeping the existing value for apptno")
#         elif street == Null:
#             print("keeping the existing value for apptno")
#         else:
#             print("Keeping the existing values for both")

            
#         city = input("\t\t" + records[0][6])
#         city,country,zipcode = ("\t\t" + records[0][7]+','+records[0][8]+' - '+str(records[0][9]))
#         phno=input("Ph No:\t\t" + records[0][10])
#         email = ("Email:\t\t" + records[0][11])




    city_new_question = "City("+ city + "): "
    city_new = input(city_new_question)
    if not city_new:
        city_new = city
    print (city, city_new)
    
    streetaddr_question = 'Street address('+full_street_address+'):'
    streetaddr_new = input(streetaddr_question)
    if not streetaddr_new:
        streetaddr_new = full_street_address
        
    
    state_question = 'State('+state+'):'
    state_new = input(state_question)
    if not state_new:
        state_new = state
        
    country_question = 'Country('+country+'):'
    country_new = input(country_question)
    if not country_new:
        country_new = country
        
    zip_question = 'Zip Code('+zipcode+'):'
    zip_new = input(zip_question)
    if not zip_new:
        zip_new = zipcode
        
    phno_question = 'Phone Number('+phno+'):'
    phno_new = input(phno_question)
    if not phno_new:
        phno_new = phno
        
    email_question = 'Email('+phno+'):'
    email_new = input(email_question)
    if not email_new:
        email_new = email





    
    
    ssn=123451007
    
    
    
    query = 'update cdw_sapp_customer \
    set full_street_address = %s, cust_city = %s, \
    cust_state = %s ,cust_country = %s, \
    cust_zip = %s, cust_phone = %s, \
    cust_email=%s \
    where ssn = %s'




    cursor.execute(query,(streetaddr_new,city_new,state_new,country_new,zip_new,phno_new,email_new, ssn,))
    
    print('Sucessfully updated')
    query = 'select * from cdw_sapp_customer where ssn = %s'
    cursor.execute(query,(ssn,))

    records = cursor.fetchall()
    print(records)

    #database.commit()

    cursor.close()   
    


    
    if len(records) == 0:
        print('Could find any account details with given input')
    else:
    
        print("SSN:\t\t"+ '***-**-'+ssn)
        print("Name:\t\t"+(records[0][1]+' '+records[0][2].title()+' '+records[0][3]))
        print("CC No:\t\t"+records[0][4])
        print("Address:\t" + records[0][5])
        print("\t\t" + records[0][6])
        print("\t\t" + records[0][7]+','+records[0][8]+' - '+str(records[0][9]))
        print("Ph No:\t\t" + records[0][10])
        print("Email:\t\t" + records[0][11])
        
        phno=records[0][10]
        apt_no='234'
        full_street_address = records[0][5]
        city=records[0][6]
        state=records[0][7]
        country = records[0][8]
        zipcode=records[0][9]
        email=records[0][11]
        phno=records[0][10]
        
        query = 'update cdw_sapp_customer \
        set full_street_address = CONCAT(%s,' ,',%s), \
        set cust_city=%s, cust_state=%s, cust_country=%s, \
        cust_zip = int(%s),cut_phone = %s,cust_email = %s \
        where substr(SSN,6,4) = %s,last_name = %s' 
        
#         query = 'update cdw_sapp_customer \
#         set full_street_address = CONCAT(5,' ,',aaa), \
#         cust_city=city, cust_state=state, cust_country=country, \
#         cust_zip = int(zip),cut_phone = phno,cust_email = email \
#         where substr(SSN,6,4) = %s,last_name = %s' 

        
        cursor.execute(query,(apt_no,full_street_address,city,state,country,zip,email,phno,lastname,))
        query = 'select * from cdw_sapp_customer where substr(SSN,6,4) = %s,last_name = %s'
        cursor.execute(query,(ssn,lastname,))

        records = cursor.fetchall()
        print(records)


    cursor.close()    
  

    
    
    
    
    
modify_customer_details()


In [ ]:
customer_details()

In [ ]:
# UPDATE table_name
# SET column1 = value1,
#  column2 = value2
# WHERE
# 	condition;


#     phno='1234'
#    full_street_address = '234 , aaaa'
#     city='weree'
#     state='AA'
#     country = 'fffff'
#     zip=12345
#     email='asdf@qqqqq.com'
#     phno='1234567'
#    ssn=123451007
#     lastname = 'camp'
#         query = 'update cdw_sapp_customer \
#         set full_street_address = CONCAT(5,' ,',aaa), \
#         cust_city=city, cust_state=state, cust_country=country, \
#         cust_zip = int(zip),cut_phone = phno,cust_email = email \
#         where substr(SSN,6,4) = %s,last_name = %s'     
    
def test():
    connection = get_database_connection()
    cursor = connection.cursor()
    full_street_address = '234 , aaaa'
    city='weree'
    state='AA'
    country = 'fffff'
    zipcode = 12345
    phno = '123-1234'
    email = 'asdf@yahoo.com'


    
    
    ssn=123451007
    
    
    
    query = 'update cdw_sapp_customer \
    set full_street_address = %s, cust_city = %s, \
    cust_state = %s ,cust_country = %s, \
    cust_zip = %s, cust_phone = %s, \
    cust_email=%s \
    where ssn = %s'




    cursor.execute(query,(full_street_address,city,state,country,zipcode,phno,email, ssn,))
    
    print('Sucessfully updated')
    query = 'select * from cdw_sapp_customer where ssn = %s'
    cursor.execute(query,(ssn,))

    records = cursor.fetchall()
    print(records)


    cursor.close()   
    
test()
#     cust_city=%s, cust_state=%s, cust_country=%s, \
#     cust_zip = %i,cut_phone = %s,cust_email = %s \

3) Used to generate a monthly bill for a credit card number for a given month and year.

In [4]:
def monthly_bill_per_year():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
    flag = True
    ssn = getpass("Enter the last 4 digits :    " )
    while flag:
        if not ssn.isdigit() or len(ssn) != 4:
            ssn =getpass('Invalid ssn, Re enter the ssn without -:    ')
        else:
            flag=False
            
    lastname = input('Enter the lastname:   ')
    
    transaction_year = input("Enter the year for transactions records:  ")
    flag=True
    while (flag):
        if not transaction_year.isdigit() or len(transaction_year)!=4:
            transaction_year = input("Enter valid year format XXXX")
        else:
            flag = False
        
    transaction_month = input("Enter the month in the form (XX): ").zfill(2)
    flag = True
    while (flag):
        if  not transaction_month.isdigit():
            transaction_month = input('Please enter valid month')
        else:
            flag=False
    query="select * from cdw_sapp_customer where substr(ssn,6,4) = %s and Last_name = %s"
    
    cursor.execute(query,(ssn,lastname,))
    
    records = cursor.fetchall()
    
    if len(records) == 0:
        print('Could find any account details with given input')
    else:    
        cust_ssn=records[0][0]   
        
        
#     query="select * from cdw_sapp_credit_card where cust_ssn = %s"
    
#     cursor.execute(query,(cust_ssn,))
    
#     records = cursor.fetchone()
#     print('\n')
#     if len(records) == 0:
#         print('Could find any account details with given input')
#     else:
#         print(records)



    


    query = 'select cc.cust_cc_no, sum(cc.transaction_value) from cdw_sapp_credit_card as cc\
    join cdw_sapp_customer as c on c.ssn = cc.cust_ssn \
    where SUBSTRING(timeid, 1, 4)  = %s and SUBSTRING(timeid, 5, 2)  = %s and c.ssn = %s'


    cursor.execute(query,(transaction_year,transaction_month,cust_ssn,))
    records = cursor.fetchall()
    print("\n List of  customer transactions on a given credit card number : ")
    for row in records:
        print(row)
    connection.close()
monthly_bill_per_year()


 List of  customer transactions on a given credit card number : 
('4210653344660822', 146.39)


4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.

In [ ]:
def monthly_bill_between_dates():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
    flag = True
        
    ssn = getpass("Enter the last 4 digits :    " )
    while flag:
        if not ssn.isdigit() or len(ssn) != 4:
            ssn =getpass('Invalid ssn, Re enter the ssn without -:    ')
        else:
            flag=False
            
    lastname = input('Enter the lastname:   ')
    
    start_date = input("Enter the start date for transactions recordsin the format XXXXXXXX:  ")
    flag=True
    while (flag):
        if not start_date.isdigit() or len(start_date)!=8:
            transaction_year = input("Enter valid year format XXXXXXXX")
        else:
            flag = False
        
    end_date = input("Enter the end date in the form (XXXXXXXX): ")
    flag = True
    while (flag):
        if not end_date.isdigit() or len(end_date)!=8:
            transaction_month = input('Please enter valid date in the format XXXXXXXX')
        else:
            flag=False
            
            
            
    query="select * from cdw_sapp_customer where substr(ssn,6,4) = %s and Last_name = %s"
    
    cursor.execute(query,(ssn,lastname,))
    
    records = cursor.fetchall()
    
    if len(records) == 0:
        print('Could find any account details with given input')
        return
    else:    
        cust_ssn=records[0][0]   
        
        
#     query="select * from cdw_sapp_credit_card where cust_ssn = %s"
    
#     cursor.execute(query,(cust_ssn,))
    
#     records = cursor.fetchone()
#     print('\n')
#     if len(records) == 0:
#         print('Could find any account details with given input')
#     else:
#         print(records)


    if start_date>=end_date:
        print('please enter valid range')
        flag=True
        while (flag):
            if not start_date.isdigit() or len(start_date)!=8:
                transaction_year = input("Enter valid year format XXXXXXXX")
            else:
                flag = False

        end_date = input("Enter the end date in the form (XXXXXXXX): ")
        flag = True
        while (flag):
            if not end_date.isdigit() or len(end_date)!=8:
                transaction_month = input('Please enter valid date in the format XXXXXXXX')
            else:
                flag=False




    


    query = 'select cc.* from cdw_sapp_credit_card as cc\
    join cdw_sapp_customer as c on c.ssn = cc.cust_ssn \
    where cc.cust_ssn = %s and cc.timeid between  %s and %s \
    order by cc.timeid desc'


    cursor.execute(query,(cust_ssn,start_date,end_date,))
    records = cursor.fetchall()
    print("\n List of  customer transactions on a given credit card number : ")
    for row in records:
        print(row)
    connection.close()
monthly_bill_between_dates()